In [12]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import numpy as np

# Load the data from "merged_training.pkl"
df = pd.read_pickle("merged_training.pkl")

# Encode the emotion labels using LabelEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['emotions_encoded'] = label_encoder.fit_transform(df['emotions'])

# Split the data into train and test sets
X = df['text']
y = df['emotions_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the list of emotion labels for later decoding
emotions = list(label_encoder.classes_)

In [14]:
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf

# Define BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the training and test data
train_encodings = tokenizer(list(X_train), truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(list(X_test), truncation=True, padding=True, return_tensors='tf')

# Load pre-trained BERT model
model_bert = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=len(emotions))

# Compile the BERT model
model_bert.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss=model_bert.compute_loss, metrics=['accuracy'])

# Train the BERT model
model_bert.fit(train_encodings, y_train, epochs=3)


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/3


AttributeError: in user code:

    File "C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 1377, in train_function  *
        return step_function(self, iterator)
    File "d:\Abhishek\Anaconda\Lib\site-packages\transformers\modeling_tf_utils.py", line 1553, in compute_loss  *
        return super().compute_loss(*args, **kwargs)
    File "C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py", line 1185, in compute_loss  **
        return self.compiled_loss(
    File "C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\compile_utils.py", line 275, in __call__
        y_t, y_p, sw = match_dtype_and_rank(y_t, y_p, sw)
    File "C:\Users\ASUS\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\compile_utils.py", line 854, in match_dtype_and_rank
        if (y_t.dtype.is_floating and y_p.dtype.is_floating) or (

    AttributeError: 'NoneType' object has no attribute 'dty

In [15]:
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, TFBertForSequenceClassification
import tensorflow as tf
import numpy as np

# Load the data from "merged_training.pkl"
df = pd.read_pickle("merged_training.pkl")

# Encode the emotion labels using LabelEncoder
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['emotions_encoded'] = label_encoder.fit_transform(df['emotions'])

# Sample a smaller subset of the data for faster experimentation
df = df.sample(frac=0.1, random_state=42)  # Adjust the fraction as needed

# Split the data into train and test sets
X = df['text']
y = df['emotions_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the list of emotion labels for later decoding
emotions = list(label_encoder.classes_)

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the text data with a reduced maximum sequence length
max_length = 64  # Adjust the maximum sequence length
X_train_tokens = tokenizer(list(X_train), padding=True, truncation=True, max_length=max_length, return_tensors='tf')
X_test_tokens = tokenizer(list(X_test), padding=True, truncation=True, max_length=max_length, return_tensors='tf')

# Convert the emotion labels to categorical format
num_classes = len(emotions)

# Load the pre-trained BERT model
model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes)

# Compile the model with a reduced learning rate
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5)  # Adjust the learning rate
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=optimizer, loss=loss, metrics=[metric])

# Train the model with reduced epochs and batch size
history = model.fit(
    X_train_tokens.data,
    y_train,
    validation_data=(X_test_tokens.data, y_test),
    epochs=1,  # Reduce the number of epochs
    batch_size=16  # Reduce the batch size
)

# Evaluate the model
loss, accuracy = model.evaluate(X_test_tokens.data, y_test, batch_size=16)  # Use the same reduced batch size
print(f'Loss: {loss}, Accuracy: {accuracy}')

# Save the model
model.save_pretrained('emotion_model')


All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


522/522 [==============================] - 1821s 3s/step - loss: 0.1292 - accuracy: 0.9327
Loss: 0.1291562020778656, Accuracy: 0.9327096343040466
